In [2]:
import sys
import os
import yaml
import zipfile
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
class Loader():
    def __init__(self, image_path = None, image_size = 256, channels = 3, batch_size = 1, split_size = 0.20, **kwargs):
        self.image_path = image_path
        self.image_size = image_size
        self.channels = channels
        self.batch_size = batch_size
        self.split_size = split_size
        self.paired_image = kwargs["paired_images"]
        self.unpaired_image = kwargs["unpaired_images"]
        
        
    def transforms(self):
        pass
    
    def data_split(self):
        pass
    
    def image_info(self):
        pass
    
    def unzip_folder(self):
        pass
    
    def extract_features(self):
        pass
    
    def create_dataloader(self):
        pass
    
    
    @staticmethod
    def plot_images():
        pass
    
    def dataset_details():
        pass    